<a href="https://colab.research.google.com/github/Leonardo1133/Predicting-COVID19-using-machine-learning/blob/master/covid19Arg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*   Autor: Leonardo Oscar Dávila 
*   Ingeniero Electrónico | Data Scientist
*   leonardo.davila1133@gmail.com
*   linkedin: https://www.linkedin.com/in/leonardooscardavila/



In [0]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
url = 'http://170.150.153.128/covid/Covid19Casos.csv'
df = pd.read_csv(url)

In [0]:
df.head()

,id_evento_caso,sexo,edad,edad_años_meses,residencia_pais_nombre,residencia_provincia_nombre,residencia_departamento_nombre,carga_provincia_nombre,fecha_inicio_sintomas,fecha_apertura,sepi_apertura,fecha_internacion,cuidado_intensivo,fecha_cui_intensivo,fallecido,fecha_fallecimiento,asistencia_respiratoria_mecanica,carga_provincia_id,origen_financiamiento,clasificacion,clasificacion_resumen,residencia_provincia_id,fecha_diagnostico,residencia_departamento_id,ultima_actualizacion
0,672064,M,52.0,Años,Argentina,Buenos Aires,Florencio Varela,Buenos Aires,2020-05-29,2019-10-29,44,NaN,NaN,NaN,NaN,NaN,NaN,6,Público,Caso descartado,Descartado,6,2020-06-01,274,2020-06-03
1,717629,F,46.0,Años,Argentina,Buenos Aires,Pergamino,Buenos Aires,2020-01-29,2020-01-31,5,NaN,NaN,NaN,NaN,NaN,NaN,6,Público,Caso descartado,Descartado,6,NaN,623,2020-06-03
2,717926,F,41.0,Años,Argentina,CABA,COMUNA 12,CABA,2020-01-23,2020-01-31,5,NaN,NaN,NaN,NaN,NaN,NaN,2,Público,Caso invalidado por epidemiología,Descartado,2,NaN,12,2020-06-03
3,718029,F,52.0,Años,Argentina,Buenos Aires,La Plata,Buenos Aires,2020-01-29,2020-01-31,5,NaN,NaN,NaN,NaN,NaN,NaN,6,Público,Caso invalidado por epidemiología,Descartado,6,NaN,441,2020-06-03
4,718055,F,34.0,Años,Argentina,CABA,SIN ESPECIFICAR,CABA,2020-01-16,2020-01-31,5,NaN,NaN,NaN,NaN,NaN,NaN,2,Privado,Caso descartado,Descartado,2,2020-02-11,0,2020-06-03


In [0]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 158962 entries, 0 to 158961
Data columns (total 25 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   id_evento_caso                    158962 non-null  int64  
 1   sexo                              158679 non-null  object 
 2   edad                              158681 non-null  float64
 3   edad_años_meses                   158962 non-null  object 
 4   residencia_pais_nombre            158962 non-null  object 
 5   residencia_provincia_nombre       158962 non-null  object 
 6   residencia_departamento_nombre    158962 non-null  object 
 7   carga_provincia_nombre            158962 non-null  object 
 8   fecha_inicio_sintomas             107270 non-null  object 
 9   fecha_apertura                    158962 non-null  object 
 10  sepi_apertura                     158962 non-null  int64  
 11  fecha_internacion                 44778 non-null   o

In [0]:
df.shape

(158962, 25)

In [0]:
df = df.loc[:,['sexo','edad','residencia_provincia_nombre','clasificacion_resumen','fallecido']]

In [0]:
print(pd.isnull(df).sum())

sexo                              283
edad                              281
residencia_provincia_nombre         0
clasificacion_resumen               0
fallecido                      147814
dtype: int64


Se eliminan datos faltantes en sexo, edad y fallecidos

In [0]:
df = df.dropna(axis=0)
df = df.dropna(axis=1)

In [0]:
print(pd.isnull(df).sum())

sexo                           0
edad                           0
residencia_provincia_nombre    0
clasificacion_resumen          0
fallecido                      0
dtype: int64


In [0]:
df['sexo'].value_counts()

F    5590
M    5523
A      15
Name: sexo, dtype: int64

In [0]:
indexNames = df[df['sexo'] == 'A' ].index
df.drop(indexNames , inplace=True)

In [0]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11113 entries, 10 to 158944
Data columns (total 5 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   sexo                         11113 non-null  object 
 1   edad                         11113 non-null  float64
 2   residencia_provincia_nombre  11113 non-null  object 
 3   clasificacion_resumen        11113 non-null  object 
 4   fallecido                    11113 non-null  object 
dtypes: float64(1), object(4)
memory usage: 520.9+ KB


In [0]:
df['clasificacion_resumen'].value_counts()

Descartado    7531
Confirmado    3203
Sospechoso     379
Name: clasificacion_resumen, dtype: int64

In [0]:
df['fallecido'].value_counts()

NO    8668
SI    1907
ND     538
Name: fallecido, dtype: int64

In [0]:
indexNames = df[df['fallecido'] == 'ND' ].index
df.drop(indexNames , inplace=True)

In [0]:
df['fallecido'].value_counts()

NO    8668
SI    1907
Name: fallecido, dtype: int64

Encoding categorical features

In [0]:
df['sexo'].replace(['M','F'],[1,0],inplace=True)
df['residencia_provincia_nombre'].replace(['Buenos Aires','CABA','Córdoba','Santa Fe','Chaco','Tucumán','Corrientes','Río Negro','Mendoza','Jujuy','Tierra del Fuego','Neuquén','Santiago del Estero','La Rioja','Entre Ríos','Misiones','Formosa','San Juan','Salta','SIN ESPECIFICAR','Santa Cruz','Chubut','San Luis','Catamarca','La Pampa'],[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24],inplace=True)
df['clasificacion_resumen'].replace(['Descartado','Sospechoso','Confirmado'],[0,1,2],inplace=True)
df['fallecido'].replace(['SI','NO'],[1,0],inplace=True)

In [0]:
df

,sexo,edad,residencia_provincia_nombre,clasificacion_resumen,fallecido
10,1,33.0,8,0,0
17,0,45.0,1,0,0
19,1,22.0,1,0,0
20,1,47.0,1,0,0
32,0,31.0,1,0,0
...,...,...,...,...,...
157825,0,80.0,1,1,0
157911,1,54.0,1,1,1
158066,1,20.0,0,1,0
158143,1,89.0,1,1,0


Construccion del Modelo de Machine Learning

In [0]:
X = np.array(df.drop(['fallecido'], 1))
y = np.array(df['fallecido'])

In [0]:
X

array([[ 1., 33.,  8.,  0.],
       [ 0., 45.,  1.,  0.],
       [ 1., 22.,  1.,  0.],
       ...,
       [ 1., 20.,  0.,  1.],
       [ 1., 89.,  1.,  1.],
       [ 1., 28.,  0.,  1.]])

In [0]:
y

array([0, 0, 0, ..., 0, 0, 0])

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [0]:
## Regresión logística
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
Y_pred = logreg.predict(X_test)
print('Precisión Regresión Logística:')
print(logreg.score(X_train, y_train))

Precisión Regresión Logística:
0.8306146572104018


In [0]:
## Support Vector Machines
svc = SVC()
svc.fit(X_train, y_train)
Y_pred = svc.predict(X_test)
print('Precisión Soporte de Vectores:')
print(svc.score(X_train, y_train))


Precisión Soporte de Vectores:
0.8281323877068558


In [0]:
## K neighbors
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train, y_train)
Y_pred = knn.predict(X_test)
print('Precisión Vecinos más Cercanos:')
print(knn.score(X_train, y_train))

Precisión Vecinos más Cercanos:
0.8554373522458629


In [166]:
## neural network
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 1), random_state=1)
clf.fit(X_train, y_train)
print('neural_network')
print(clf.score(X_train, y_train))

neural_network
0.8335697399527187
